In [1]:
import os
os.getcwd()  # Ensure the current working directory is set correctly

'c:\\rutgers\\Data Wrangling\\data_wrangling_msds597\\src'

### Step 1: Load and clean product data from CSV

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("../dat/wayfair_bs4_products.csv")

df.head()

,title,price,reviews,star_rating,url
0,"63"" L Shaped Office Executive Standing Desk wi...",$231.99,(239),4.0,https://www.wayfair.com/furniture/pdp/hokku-de...
1,Dediu Farmhouse Computer Desk With Charging St...,$183.99,(173),4.5,https://www.wayfair.com/furniture/pdp/august-g...
2,Koree Standing & Height-Adjustable Desks,$131.99,(64),5.0,https://www.wayfair.com/furniture/pdp/inbox-ze...
3,Caidence Computer Desk,$699.99,(11),3.5,https://www.wayfair.com/furniture/pdp/george-o...
4,Channay Writing Desk,$309.99,(2),4.0,https://www.wayfair.com/furniture/pdp/wade-log...


In [16]:
# Clean the 'price' column: remove dollar signs and convert to float
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

# Remove parentheses and drop NaN before converting to int
df['reviews'] = df['reviews'].astype(str).str.replace('[()]', '', regex=True)
df = df[df['reviews'].str.lower() != 'nan']
df['reviews'] = df['reviews'].astype(int)


# Add a 'page' column to indicate which page the product came from (6 items per page)
df['page'] = df.index // 6 + 1

df.head()

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Shang Yi Lin\AppData\Local\Temp\ipykernel_13968\1086219840.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)


,title,price,reviews,star_rating,url,page
0,"63"" L Shaped Office Executive Standing Desk wi...",231.99,239,4.0,https://www.wayfair.com/furniture/pdp/hokku-de...,1
1,Dediu Farmhouse Computer Desk With Charging St...,183.99,173,4.5,https://www.wayfair.com/furniture/pdp/august-g...,1
2,Koree Standing & Height-Adjustable Desks,131.99,64,5.0,https://www.wayfair.com/furniture/pdp/inbox-ze...,1
3,Caidence Computer Desk,699.99,11,3.5,https://www.wayfair.com/furniture/pdp/george-o...,1
4,Channay Writing Desk,309.99,2,4.0,https://www.wayfair.com/furniture/pdp/wade-log...,1


In [37]:
import plotly.express as px

fig = px.histogram(
    df,
    x="price",
    nbins=20,
    title="Price Distribution",
    labels={"price": "Price ($)"},
    marginal="box"  # Optional: adds box plot on side
)
fig.update_layout(bargap=0.1)
fig.show()


In [38]:
fig = px.histogram(
    df,
    x="star_rating",
    title="Star Rating Distribution",
    labels={"star_rating": "Star Rating"},
    category_orders={"star_rating": sorted(df["star_rating"].dropna().unique())}
)
fig.update_layout(bargap=0.2)
fig.show()


In [43]:
fig = px.histogram(
    df,
    x="reviews",
    nbins=100,  # more bins = finer histogram
    title="Reviews Distribution (Fine-grained)",
    labels={"reviews": "Number of Reviews"}
)
fig.update_layout(bargap=0.01)  # reduce gap between bars
fig.show()


In [20]:
page_summary = df.groupby('page').agg({
    'price': 'mean',
    'reviews': 'mean',
    'star_rating': 'mean'
}).reset_index()


In [47]:
fig = px.line(
    page_summary,
    x="page",
    y="price",
    markers=True,
    title="Average Price per Page",
    labels={"page": "Page Number", "price": "Average Price ($)"}
)


tickvals = page_summary['page'].unique()[::10]

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=tickvals,
        ticktext=[str(v) for v in tickvals]
    ),
    hovermode="x unified"
)

fig.show()


In [49]:
fig = px.line(
    page_summary,
    x="page",
    y="star_rating",
    markers=True,
    title="Average Star Rating per Page",
    labels={"page": "Page Number", "star_rating": "Average Star Rating"}
)

tickvals = page_summary['page'].unique()[::10]

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=tickvals,
        ticktext=[str(v) for v in tickvals]
    ),
    yaxis=dict(
        range=[3.5, 5]  # 👈 focus on the upper rating range
    ),
    hovermode="x unified"
)

fig.show()


In [25]:
# Set folder path and target date
folder_path = "../dat"
target_date = "20250429"

# List all CSV files in the folder
all_files = os.listdir(folder_path)

# Filter files that match the target date
target_files = [f for f in all_files if f.endswith(".csv") and target_date in f]

# Read and concatenate all matching files
df_list = []
for file in target_files:
    full_path = os.path.join(folder_path, file)
    temp_df = pd.read_csv(full_path)  # ← avoid overwriting existing 'df'
    temp_df["source_file"] = file
    df_list.append(temp_df)

# Combine into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

print(f"Loaded {len(combined_df)} rows from {len(target_files)} files.")
combined_df.head()


Loaded 100 rows from 10 files.


,timestamp,url,price,time_spent_sec,source_file
0,2025-04-29 15:22:54,https://www.wayfair.com/furniture/pdp/hokku-de...,$539.99,39.45,wayfair_price_tracking_20250429_1546.csv
1,2025-04-29 15:23:33,https://www.wayfair.com/furniture/pdp/august-g...,$259.99,28.65,wayfair_price_tracking_20250429_1546.csv
2,2025-04-29 15:24:02,https://www.wayfair.com/furniture/pdp/inbox-ze...,NaN,188.73,wayfair_price_tracking_20250429_1546.csv
3,2025-04-29 15:27:11,https://www.wayfair.com/furniture/pdp/george-o...,NaN,325.68,wayfair_price_tracking_20250429_1546.csv
4,2025-04-29 15:32:36,https://www.wayfair.com/furniture/pdp/wade-log...,NaN,231.67,wayfair_price_tracking_20250429_1546.csv


In [26]:
# --- 3. Clean and format data ---
# Clean price column (remove $ and convert to float)
combined_df["price"] = combined_df["price"].replace('[\$,]', '', regex=True).astype(float)

# Convert timestamp to datetime
combined_df["timestamp"] = pd.to_datetime(combined_df["timestamp"])

<>:3: SyntaxWarning:

invalid escape sequence '\$'

<>:3: SyntaxWarning:

invalid escape sequence '\$'

C:\Users\Shang Yi Lin\AppData\Local\Temp\ipykernel_13968\1327830988.py:3: SyntaxWarning:

invalid escape sequence '\$'



In [28]:
# Only merge 'title' column from df
merged_df = pd.merge(
    combined_df,
    df[['url', 'title']],  # Use the original df to get the 'title' column
    on='url',
    how='left'
)


merged_df.head()

,timestamp,url,price,time_spent_sec,source_file,title
0,2025-04-29 15:22:54,https://www.wayfair.com/furniture/pdp/hokku-de...,539.99,39.45,wayfair_price_tracking_20250429_1546.csv,"63"" L Shaped Office Executive Standing Desk wi..."
1,2025-04-29 15:23:33,https://www.wayfair.com/furniture/pdp/august-g...,259.99,28.65,wayfair_price_tracking_20250429_1546.csv,Dediu Farmhouse Computer Desk With Charging St...
2,2025-04-29 15:24:02,https://www.wayfair.com/furniture/pdp/inbox-ze...,NaN,188.73,wayfair_price_tracking_20250429_1546.csv,Koree Standing & Height-Adjustable Desks
3,2025-04-29 15:27:11,https://www.wayfair.com/furniture/pdp/george-o...,NaN,325.68,wayfair_price_tracking_20250429_1546.csv,Caidence Computer Desk
4,2025-04-29 15:32:36,https://www.wayfair.com/furniture/pdp/wade-log...,NaN,231.67,wayfair_price_tracking_20250429_1546.csv,Channay Writing Desk


In [30]:
import plotly.graph_objects as go

# Step 1: prepare all traces, one for each product
titles = merged_df['title'].dropna().unique()
fig = go.Figure()

for i, title in enumerate(titles):
    product_data = merged_df[merged_df['title'] == title]
    fig.add_trace(go.Scatter(
        x=product_data['timestamp'],
        y=product_data['price'],
        mode='lines+markers',
        name=title,
        visible=(i == 0)  # Only show the first product initially
    ))

# Step 2: Create dropdown menu
dropdown_buttons = [
    dict(
        method="update",
        label=title,
        args=[{"visible": [t == i for t in range(len(titles))]},
              {"title": f"Price Trend for: {title}"}]
    )
    for i, title in enumerate(titles)
]

# Step 3: Update layout with dropdown
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title=f"Price Trend for: {titles[0]}",
    xaxis_title="Time",
    yaxis_title="Price ($)",
    hovermode="x unified"
)

fig.show()


In [31]:
# Step 1: prepare traces for time_spent_sec
titles = merged_df['title'].dropna().unique()
fig = go.Figure()

for i, title in enumerate(titles):
    product_data = merged_df[merged_df['title'] == title]
    fig.add_trace(go.Scatter(
        x=product_data['timestamp'],
        y=product_data['time_spent_sec'],
        mode='lines+markers',
        name=title,
        visible=(i == 0)  # Show only the first product by default
    ))

# Step 2: create dropdown buttons
dropdown_buttons = [
    dict(
        method="update",
        label=title,
        args=[{"visible": [t == i for t in range(len(titles))]},
              {"title": f"Time Spent Trend for: {title}"}]
    )
    for i, title in enumerate(titles)
]

# Step 3: apply layout
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title=f"Time Spent Trend for: {titles[0]}",
    xaxis_title="Time",
    yaxis_title="Time Spent (Seconds)",
    hovermode="x unified"
)

fig.show()


In [34]:
import re
from datetime import datetime


# Read log file
with open("../dat/wayfair_scrape_log.txt", "r") as f:
    raw_log = f.read()

# Split by each log run
log_blocks = raw_log.strip().split("========================================")

records = []

for block in log_blocks:
    lines = block.strip().split("\n")
    if len(lines) < 2:
        continue

    timestamp = re.search(r"\[(.*?)\]", lines[0]).group(1)
    timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")

    total = int(re.search(r"Total URLs: (\d+)", block).group(1))
    success = int(re.search(r"Successful fetches: (\d+)", block).group(1))
    failed = int(re.search(r"Failed fetches: (\d+)", block).group(1))
    rate = float(re.search(r"Success rate: ([\d\.]+)", block).group(1))
    avg_time = float(re.search(r"Average time per URL: ([\d\.]+)", block).group(1))

    records.append({
        "timestamp": timestamp,
        "total": total,
        "success": success,
        "failed": failed,
        "success_rate": rate,
        "avg_time_sec": avg_time
    })

log_df = pd.DataFrame(records)
log_df = log_df.sort_values("timestamp")  # Optional: sort by time
log_df.head()


,timestamp,total,success,failed,success_rate,avg_time_sec
2,2025-04-29 15:46:02,10,7,3,70.0,138.85
0,2025-04-29 16:02:34,10,7,3,70.0,98.37
3,2025-04-29 16:25:07,10,7,3,70.0,84.09
4,2025-04-29 17:25:05,10,7,3,70.0,95.61
5,2025-04-29 18:21:48,10,7,3,70.0,74.61


In [35]:
fig = px.line(
    log_df,
    x="timestamp",
    y="avg_time_sec",
    markers=True,
    title="Average Time per URL (by Run)",
    labels={"timestamp": "Time", "avg_time_sec": "Avg. Time (sec)"}
)
fig.update_layout(hovermode="x unified")
fig.show()


In [36]:
fig = px.line(
    log_df,
    x="timestamp",
    y="success_rate",
    markers=True,
    title="Success Rate Over Time",
    labels={"timestamp": "Time", "success_rate": "Success Rate (%)"}
)
fig.update_layout(hovermode="x unified", yaxis_range=[0, 100])
fig.show()
